# Ensure no data leakage across experiment steps (baseline training, offline ST, final finetuning)
Also ensure that patching does not leak samples (i.e. if limiting to 6 original samples, doing 4-patching and limit_train_samples=6*4, it should only include the same samples as with no patching and limit_train_samples=6)

## Get unique origin sample ids used within offlineST experiment

In [1]:
from src.experiments.offlinest_experiment import (
    OfflineSTExperiment,
    OfflineSTExperimentArgs,
)
from src.args.yaml_config import YamlConfig

yaml_config = YamlConfig().config
exp = OfflineSTExperiment(
    OfflineSTExperimentArgs(
        sam_model="vit_b",
        experiment_id="offline_st",
        wandb_tags=[],
        batch_size=32,
        pseudo_labels_dir="teacher_8_samples_not_thresholded",
        gt_limit=8,
        labelled_ratio=0.25
    ).model_dump(),
    yaml_config,
)

dl = exp._create_dataloader("train")

sample = {}
n = 0
for i, batch in enumerate(dl):
    print(f"Batch {i}/{len(dl)}\r", end="")
    for s in batch.metadata["sample_metadata"]:
        if s["origin_dataset"] not in sample:
            sample[s["origin_dataset"]] = list()
        sample[s["origin_dataset"]].append(s)
        n += 1
    if n > 1000:
        break
print()

sample_ids = [f"{s['origin_class']}_{s['source_idx']}" for s in sample["gt"]]
set(sample_ids)

/dhc/home/tobias.fiedler/conda3/envs/autosam/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/dhc/home/tobias.fiedler/conda3/envs/autosam/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/dhc/home/tobias.fiedler/conda3/envs/autosam/lib/python3.10/site-packages/pydantic/main.py:292: UserWarning: Pydantic serializer warnings:
  Expected `float` but got `int` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
/dhc/home/tobias.fiedler/mp-lippert-experiments/src/models/segment_anything/build_sam.py:118: FutureWarning: You are using `torch.load` with `weights_only=False` (the current de

ImageNet pretrained weights for HarDNet68 is loaded
Batch 31/5122


{'ARIASample_12',
 'ARIASample_78',
 'ARIASample_87',
 'ARIASample_90',
 'ChaseDb1Sample_28',
 'DriveSample_9',
 'STARESample_3',
 'STARESample_7'}

# Get unique origin sample ids used within baseline training and final finetuning (both equal)

In [3]:
from typing import Literal
from src.experiments.multi_ds_vessel_experiment import (
    MultiDsVesselExperiment,
    MultiDSVesselExperimentArgs,
)
from src.args.yaml_config import YamlConfig

yaml_config = YamlConfig().config

def get_sample_idxs(gt_limit: int, patching: Literal[4,16]|None=None):
    exp = MultiDsVesselExperiment(
        MultiDSVesselExperimentArgs(
            sam_model="vit_b",
            wandb_tags=[],
            experiment_id="multi_vessel_ds",
            limit_train_samples=gt_limit * (patching if patching is not None else 1),
            batch_size=3,
            patch_samples=patching,
        ).model_dump(),
        yaml_config,
    )


    dl = exp._create_dataloader("train")

    unique_origin_sample_idx = set()
    total = 0
    for i, batch in enumerate(dl):
        print(f"Batch {i}\r", end="")
        for s in batch.metadata["sample_metadata"]:
            total += 1
            unique_origin_sample_idx.add(s["origin_sample_idx"])
    return unique_origin_sample_idx

In [4]:
# print("GT 6, 4 patching", get_sample_idxs(6, 4))
# print("GT 6, 16 patching", get_sample_idxs(6, 16))
# print("GT 6, no patching", get_sample_idxs(6, None))

print("\n\n\n\nGT 8, 4 patching", get_sample_idxs(8, 4))
#print("\n\n\n\nGT 8, 16 patching", get_sample_idxs(8, 16))
#print("\n\n\n\nGT 8, no patching", get_sample_idxs(8, None))

/dhc/home/tobias.fiedler/conda3/envs/autosam/lib/python3.10/site-packages/pydantic/main.py:292: UserWarning: Pydantic serializer warnings:
  Expected `float` but got `int` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
/dhc/home/tobias.fiedler/mp-lippert-experiments/src/models/segment_anything/build_sam.py:118: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serializati

ImageNet pretrained weights for HarDNet68 is loaded
Batch 10



GT 8, 4 patching {'ARIADataset_78', 'STAREDataset_7', 'ARIADataset_90', 'ARIADataset_87', 'ChaseDb1Dataset_28', 'DriveDataset_9', 'ARIADataset_12', 'STAREDataset_3'}
